In [18]:
import ee
import google.auth
import math

In [7]:
# this is needed to Successfully save authorization token. from ee.Authenticate()
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [8]:
ee.Authenticate()



Successfully saved authorization token.


In [11]:
ee.Initialize()

In [12]:
def mask_sentinel2_clouds(image: ee.Image) -> ee.Image:
    CLOUD_BIT = 10
    CIRRUS_CLOUD_BIT = 11
    bit_mask = (1 << CLOUD_BIT) | (1 << CIRRUS_CLOUD_BIT)
    mask = image.select("QA60").bitwiseAnd(bit_mask).eq(0)
    return image.updateMask(mask)
    
def get_input_image(year: int, default_value: float = 1000.0) -> ee.Image:
  return (
      ee.ImageCollection("COPERNICUS/S2_HARMONIZED")        # Sentinel-2 images
      .filterDate(f"{year}-1-1", f"{year}-12-31")           # filter by year
      .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 20))  # filter cloudy images 
      .map(mask_sentinel2_clouds)  # mask/hide cloudy pixels
      .select("B.*")          # select all bands starting with B
      .median()               # median of all non-cloudy pixels
      .unmask(default_value)  # default value for masked pixels
      .float()                # convert to float32
  )

import folium

image = get_input_image(2020)
vis_params = {
  "min": 0,
  "max": 3000,
  "bands": ["B4", "B3", "B2"],
}
folium.Map(
    location=(30.95, -110.29),
    zoom_start=13,
    tiles=image.getMapId(vis_params)["tile_fetcher"].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
)

In [14]:
import tensorflow as tf
print(tf.__version__)

2023-03-14 13:20:47.137746: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.11.0


In [15]:
import folium
print(folium.__version__)

0.14.0


In [16]:
WorldCover10m = ee.ImageCollection("ESA/WorldCover/v100")

In [20]:
SURF_REF_SEN2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")

In [19]:
# bands used for prediction
BANDS = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']


In [111]:
def cloudmask_surfRefSen2(image):
    opaqueClouds_mask = 1 << 10
    cirrusClouds_mask =1 << 11
    bit_mask =opaqueClouds_mask |  cirrusClouds_mask
    qa = image.select('QA60')
    mask = qa.bitwiseAnd(bit_mask).eq(0)
    return image
    return image.updateMask(mask).select(BANDS).divide(10000)

# ee.image.select()  supports regex
image = SURF_REF_SEN2.filterDate('2018-01-01', '2018-12-31').map(cloudmask_surfRefSen2).select("B.*").filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 20)).median().unmask(1000.0).float()



In [113]:
vis_params = {
  "min": 0,
  "max": 3000,
  "bands": ["B4", "B3", "B2"],
}

mapid = image.getMapId(vis_params)
map = folium.Map(location=[50., 31], zoom_start=13)

folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='median composite',
  ).add_to(map)
map.add_child(folium.LayerControl())
map